<a href="https://colab.research.google.com/github/Inkyu-Yang356/machine-learning-practice/blob/main/250416_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블

## 랜덤포레스트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)  # n_jobs=-1, 모든 CPU 코어
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)  # 데이터 모델과 훈련데이터 교차검증, 학습데이터 반환
# 테스트 데이터 뿐만 아니라 훈련 데이터 결과까지 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)  # 도수, 당도, 산도

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) # 부트스트랩 샘플에 포함되지 않는 남는 샘플로 평가. Out of Bag

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)  # 모든 CPU 코어
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)  # 데이터 모델과 훈련데이터 교차검증, 학습데이터 반환
# 테스트 데이터 뿐만 아니라 훈련 데이터 결과까지 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)  # 특성 중요도

[0.20183568 0.52242907 0.27573525]


## 그라디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)  # 데이터 모델과 훈련데이터 교차검증, 학습데이터 반환
# 테스트 데이터 뿐만 아니라 훈련 데이터 결과까지 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)  # 데이터 모델과 훈련데이터 교차검증, 학습데이터 반환
# 테스트 데이터 뿐만 아니라 훈련 데이터 결과까지 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)  # 특성 중요도

[0.15887763 0.6799705  0.16115187]


## 히스토그램 기반 부스팅

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)  # 데이터 모델과 훈련데이터 교차검증, 학습데이터 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.11/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [ ]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 문제 : 새로운 데이터셋으로 RandomForest, XGBoost, LightGBM 성능 비교

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer() # array
c_data = cancer.data  # X
c_target = cancer.target  # y

# 이후 코드는 Wine 데이터셋과 동일하게 진행

In [ ]:
c_data.shape

(569, 30)

In [ ]:
c_target.shape

(569,)

In [ ]:
# 데이터셋 분류
train_X, test_X, train_y, test_y = train_test_split(c_data, c_target, test_size = 0.2, random_state=42)

In [ ]:
# 랜덤포레스트 모델 정의
rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
print(train_X.shape, test_X.shape)

(455, 30) (114, 30)


In [ ]:
# 랜덤포레스트 교차 검증 및 교차 검증 각 폴드의 학습 데이터 점수 평균, 각 폴드의 테스트 데이터 점수 평균
score = cross_validate(rf_model, train_X, train_y, return_train_score=True, n_jobs=-1)

In [ ]:
# 실제 모델 학습
rf_model.fit(train_X, train_y)

# 실제 모델 평가
rf_model.score(test_X, test_y)

0.9649122807017544

In [ ]:
# XGBoost 모델 정의
xgb_model = XGBClassifier(tree_method='hist', random_state=42)
# 교차 검증
scores = cross_validate(xgb_model, train_X, train_y, return_train_score=True, n_jobs=-1)
# 교차 검증 평가 점수
print(np.mean(scores['train_score']) ,np.mean(scores['test_score']))

1.0 0.9626373626373628


In [ ]:
# LightBGM 모델
lgb_model = LGBMClassifier(random_state=42)
# 교차 검증
scores = cross_validate(lgb_model, train_X, train_y, return_train_score=True, n_jobs=-1)
# 교차 검증 평가 점수
print(np.mean(scores['train_score']) ,np.mean(scores['test_score']))

0.964835164835165
